1. [x] 자료 불러오기
2. [x] 자료 1차 가공
  + [ ] A.주요단어 확인
  + [x] B.OOO
3. [ ] 분석1 : 주요 키워드 추출
  + [ ] ~~A.TF-IDF~~
  + [x] B.TextRank
  + [ ] ~~C.TopicRank~~
  + [x] D.TAKE!
  + [ ] E.BERTopic
4. [ ] 분석2 : 특정 범주로 매칭(Zero-shot)
  + [ ] A.구분1 : [역량, 태도,]
  + [ ] B.구분2 : [인간관계, 태도,]
5. [ ] 분석3 : 기술된 내용에 대한 평가(감정분류)
  + [ ] A.모델 : OOO


In [1]:
!pip install trafilatura -qqq
!pip install summa -qqq
!pip install git+https://github.com/smirnov-am/pytopicrank.git#egg=pytopicrank 
!pip install git+https://github.com/LIAAD/yake
!pip install keyBERT -qqq
!pip install topicrankpy -qqq
!pip install git+https://github.com/ssut/py-hanspell.git
!pip install sentence_transformers -qqq
!pip install customized_konlpy -qqq

     |████████████████████████████████| 1.0 MB 26.0 MB/s 
     |████████████████████████████████| 140 kB 83.3 MB/s 
     |████████████████████████████████| 837 kB 64.8 MB/s 
     |████████████████████████████████| 412 kB 50.7 MB/s 
     |████████████████████████████████| 195 kB 74.1 MB/s 
     |████████████████████████████████| 292 kB 55.6 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.13 which is incompatible.
aiohttp 3.8.3 requires charset-normalizer<3.0,>=2.0, but you have charset-normalizer 3.0.1 which is incompatible.
     |████████████████████████████████| 54 kB 890 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/smirnov-am/pytopicrank.git to /tmp/pip-install-1qeto22k/pytop

In [2]:
import numpy as np
import itertools
import pandas as pd

# 한글정제 KoNLPy
from ckonlpy.tag import Twitter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

import nltk
import trafilatura
import numpy as np
import itertools
import pandas as pd

# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')

from itertools import islice
from tqdm.notebook import tqdm
from re import sub

In [ ]:
from tqdm import trange

## 1.자료 불러오기

In [ ]:
df = pd.read_excel('./20221207_ottmilk_review_data.xlsx',0)
# array_text_ = list(df.REVIEW.sample(frac = 1))

# df = pd.read_excel('./20221207_ottmilk_review_data.xlsx',0)
# new_header = df.iloc[0] #grab the first row for the header
# df = df[1:] #take the data less the header row
# df.columns = new_header #set the header row as the df header
# array_text_ = list(df.평가근거)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216 entries, 1 to 216
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   No                216 non-null    object
 1   그룹입사일_근무년월        216 non-null    object
 2   본부                216 non-null    object
 3   공장                216 non-null    object
 4   팀                 216 non-null    object
 5   파트                216 non-null    object
 6   직급                216 non-null    object
 7   호봉                216 non-null    object
 8   19년               198 non-null    object
 9   20년               199 non-null    object
 10  21년               213 non-null    object
 11  전형절차              157 non-null    object
 12  인적성 시행유무          216 non-null    object
 13  최종 면접관            138 non-null    object
 14  최종선발사유            146 non-null    object
 15  추천자               41 non-null     object
 16  전형 자료보관 유무        137 non-null    object
 17  평가자             

In [ ]:
from hanspell import spell_checker

array_text_2 = []
for i in trange(len(array_text_)):
    try:
        spelled_sent = spell_checker.check(array_text_[i])
        hanspell_sent = spelled_sent.checked
        array_text_2.append(hanspell_sent)
    except:
        array_text_2.append('')


100%|██████████| 216/216 [00:41<00:00,  5.23it/s]


In [ ]:
array_text_[:5]

['21년 10월 사내 흡연 건으로 징계(감봉) 이력',
 '최소한의 기본 업무만 진행하고 교대 시간 임박한 대의 검사 업무는 다음직으로 미루는 경우 발생',
 '부여된 업무는 진행하나 그외 부가적인 업무나 협조(타팀,타파트)가 미흡',
 '- 근무 시간 중 업무와 관련없는 핸드폰 사용이 잦음\n- 출근 후 업무 수행 중이 아님에도 연락이 안됨\n- 동일 업무에 대해 타 직원보다 시간이 많이 소요됨.\n- 야간 근무 시 타 직원보다 잦은 연락으로 동료 근무자 피로도 상승',
 '- 근무 역량 부족으로 설비의 트러블이 잦음\n- 교육을 하여도 개선이 잘 이루어 지지 않음']

In [ ]:
array_text_2[:5]

['21년 10월 사내 흡연 건으로 징계(감봉) 이력',
 '최소한의 기본 업무만 진행하고 교대 시간 임박한 대의 검사 업무는 다음직으로 미루는 경우 발생',
 '부여된 업무는 진행하나 그 외 부가적인 업무나 협조(타팀, 타 아파트)가 미흡',
 '- 근무 시간 중 업무와 관련 없는 핸드폰 사용이 잦음- 출근 후 업무 수행 중이 아님에도 연락이 안 됨- 동일 업무에 대해 타 직원보다 시간이 많이 소요됨.- 야간 근무 시 타 직원보다 잦은 연락으로 동료 근무자 피로도 상승',
 '- 근무 역량 부족으로 설비의 트러블이 잦음- 교육을 하여도 개선이 잘 이루어지지 않음']

##  2자료 1차 가공

In [ ]:
###############
# Text 정제
#####

twitter = Twitter()

# 사전에 고유 명사 추가
twitter.add_dictionary('상대적', 'Noun')
twitter.add_dictionary('절대적', 'Noun')
twitter.add_dictionary('지속적', 'Noun')
twitter.add_dictionary('수동적', 'Noun')
twitter.add_dictionary('소극적', 'Noun')
twitter.add_dictionary('부정적', 'Noun')
twitter.add_dictionary('자발적', 'Noun')
twitter.add_dictionary('방어적', 'Noun')
twitter.add_dictionary('파트장', 'Noun')
twitter.add_dictionary('입사', 'Noun')
twitter.add_dictionary('상대평가', 'Noun')
twitter.add_dictionary('크로네스', 'Noun')
twitter.add_dictionary('숙련도', 'Noun')
twitter.add_dictionary('안전사고', 'Noun')
twitter.add_dictionary('생산성', 'Noun')
twitter.add_dictionary('검사원', 'Noun')

twitter.add_dictionary('지원팀', 'Noun')
twitter.add_dictionary('전환배치', 'Noun')
twitter.add_dictionary('지속성', 'Noun')
twitter.add_dictionary('부족', 'Noun')
twitter.add_dictionary('액상', 'Noun')
twitter.add_dictionary('입사', 'Noun')

twitter.add_dictionary('초창기', 'Noun')
twitter.add_dictionary('근태', 'Noun')
twitter.add_dictionary('태도', 'Noun')
twitter.add_dictionary('자세', 'Noun')

twitter.add_dictionary('클레임', 'Noun')
twitter.add_dictionary('불이행', 'Noun')
twitter.add_dictionary('기술력', 'Noun')
twitter.add_dictionary('정교함', 'Noun')

twitter.add_dictionary('입사', 'Noun')
twitter.add_dictionary('공방', 'Noun')
twitter.add_dictionary('트러블', 'Noun')



/usr/local/lib/python3.7/dist-packages/konlpy/tag/_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [ ]:
tokenized_text = []

for i in trange(len(array_text_2)):
    spelled_sent = spell_checker.check(array_text_2[i])
    txt_ = spelled_sent.checked
    tokenized_doc = twitter.pos(txt_, norm=True, stem=True)
    tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if (word[1] == 'Noun') | (word[1] == 'Verb') | (word[1] == 'Adjective')])
    tokenized_text.append(tokenized_nouns)

100%|██████████| 216/216 [00:42<00:00,  5.03it/s]


In [ ]:
i = 65
tokenized_text[i], array_text_2[i]

('성희롱 문제 인사 처벌 정직 개월 로 등급 부여 함', '2021년 성희롱 문제로 인사 처벌(정직 1개월)로 C 등급 부여함')

In [ ]:
tokenized_text[:5]

NameError: ignored

In [ ]:
# !pip install torch --upgrade -qqq
# !pip install Pororo -qqq
!pip install --upgrade torch torchvision

     |████████████████████████████████| 256 kB 32.1 MB/s 
     |████████████████████████████████| 42.4 MB 1.3 MB/s 
     |████████████████████████████████| 4.2 MB 53.3 MB/s 
     |████████████████████████████████| 1.5 MB 56.1 MB/s 
     |████████████████████████████████| 1.7 MB 66.2 MB/s 
     |████████████████████████████████| 3.1 MB 49.9 MB/s 
     |████████████████████████████████| 5.9 MB 58.1 MB/s 
     |████████████████████████████████| 1.2 MB 45.8 MB/s 
     |████████████████████████████████| 79 kB 8.5 MB/s 
     |████████████████████████████████| 468 kB 64.9 MB/s 
     |▋                               | 13.3 MB 23.2 MB/s eta 0:00:32ERROR: Exception:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/base_command.py", line 180, in _main
    status = self.run(options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/req_command.py", line 199, in wrapper
    return func(self, options, args)
  File "/usr/local/li

In [ ]:
from pororo import Pororo

ModuleNotFoundError: ignored

In [ ]:
# 평가내용 분류
from pororo import Pororo
zsl = Pororo(task="zero-topic", lang="ko")

ModuleNotFoundError: ignored

In [ ]:
reason_list = ['개인태도', '직무역량', '휴직', '문제없음']

reason_1st = []
reason_row = []

for i in trange(len(tokenized_text)):
    try:
        temp = zsl(tokenized_text[i], reason_list)
        reason_row.append(temp)
        reason_1st.append(max(temp, key=temp.get))
    except:
        reason_row.append('')
        reason_1st.append('')

100%|██████████| 216/216 [00:00<00:00, 1768.92it/s]


In [ ]:
df['평가근거_2'] = array_text_2
df['평가근거_3'] = tokenized_text
df['평가근거_구분'] = reason_1st
df['평가근거_구분상세'] = reason_row
df['issu_keyword'] = TexrRank_list

In [ ]:
writer = pd.ExcelWriter('./_kurly_review.xlsx')
df.to_excel(writer, 'Sheet1')
writer.save()

In [ ]:
tokenized_text[12]

'동일 직급 상대평가 근무 태도 불량 근태 불량'

In [ ]:
reason_list = ['개인태도', '직무역량', '휴직', '문제없음']
zsl(tokenized_text[11], reason_list)

AttributeError: ignored

### A.주요단어 확인

## 3.분석1 : 주요 키워드 추출

### TextRank

In [ ]:
# from summa import keywords
# for j in range(len(array_text)):
#     print("Keywords of article", str(j+1), "\n", (keywords.keywords(array_text[j], words=5)).split("\n"))

In [ ]:
tokenized_text[:3]

['입 사하 에지 충전기 담당 있다 동료 들보 업무 습득 능력 상대적 느리다 동료 들다 낮다 평가 받다',
 '평상시 업무 불평 불만 많다 잦다 신체 문제 호소 주변 동료 들다 로부터 신뢰 저하',
 '상대평가 의하다 평가 생산성 문제 없다']

In [ ]:
from summa import keywords

TexrRank_list = []
for j in trange(len(tokenized_text)):
    try:
        temp = keywords.keywords(tokenized_text[j], words=3).split("\n")
        TexrRank_list.append(temp)
    except:
        TexrRank_list.append('')


100%|██████████| 216/216 [00:00<00:00, 554.56it/s]


In [ ]:
TexrRank_list

[['흡연', '징계 감봉'],
 ['업무', '미루다 경우'],
 ['업무', '아파트 가다'],
 ['업무', '근무', '연락'],
 ['지지', '역량 부족'],
 ['근무', '이행 하다'],
 ['점수', '작업 실수'],
 ['업무', '변화 변경'],
 ['작업', '업무 집중'],
 ['이해력', '실무 자로'],
 ['보전 대한 스킬'],
 ['직급', '전문성 숙련도'],
 ['불량', '직급 상대평가'],
 ['직급', '전문성 숙련도'],
 ['징계', '직급 상대평가'],
 ['직급', '전문성 숙련도'],
 ['직급', '전문성 숙련도'],
 ['직급', '인쇄 징계'],
 ['상대평가'],
 ['직급', '전문성 숙련도'],
 ['불량', '직급 상대평가'],
 ['직급', '지시 불'],
 ['직급', '전문성 숙련도'],
 ['에러', '직급 상대평가'],
 ['있다', '수준 상향'],
 ['지속', '가다 단계'],
 ['높다', '임 징계'],
 ['주도 전문성 상향'],
 ['징계', '대비 전문'],
 ['주도 전문성 상향'],
 ['함양', '대한 깊이'],
 ['력', '설비 관심'],
 ['상향', '높다 지속적'],
 ['수준', '성장 더디다'],
 ['주도', '관련 징계'],
 ['초창기', '평가 부여'],
 ['부족', '의식', '처리'],
 ['부족', '기준 준수'],
 ['희망', '들다 업무'],
 ['함', '인하다 미만'],
 ['등급', '본부', '문제'],
 ['등급', '본부', '문제'],
 ['등급', '본부', '문제'],
 ['불만', '성향 근무'],
 ['등급', '본부', '문제'],
 ['등급', '본부', '문제'],
 ['등급', '본부', '문제'],
 ['성과', '무능력 미진'],
 ['함'],
 ['등급', '본부', '문제'],
 ['등급', '본부', '문제'],
 ['등급', '본부', '문제'],
 ['등급', '본부', '문제'],
 ['등급', '본

### ~~TopicRank~~

In [ ]:
from pytopicrank import TopicRank
for j in range(len(array_text)):
    tr = TopicRank(array_text[j])
    print("Keywords of article", str(j+1), "\n", tr.get_top_n(n=5, extract_strategy='first'))

ModuleNotFoundError: ignored

In [ ]:
from pytopicrank import TopicRank

TexrRank_list = []
for j in trange(len(tokenized_text)):
    try:
        temp = TopicRank(tokenized_text[j])
        TexrRank_list.append(temp)
    except:
        TexrRank_list.append('')

ModuleNotFoundError: ignored

### YAKE!

In [ ]:
from yake import KeywordExtractor
kw_extractor = KeywordExtractor(lan="en", n=1, top=5)
for j in range(len(array_text)):
    keywords = kw_extractor.extract_keywords(text=array_text[j])
    keywords = [x for x, y in keywords]
    print("Keywords of article", str(j+1), "\n", keywords)

NameError: ignored

In [ ]:
kw_extractor = KeywordExtractor(lan="en", n=1, top=3)

YAKE_list= []
for j in trange(len(tokenized_text)):
    try:
        keywords = kw_extractor.extract_keywords(text=tokenized_text[j])
        temp = [x for x, y in keywords]
        YAKE_list.append(temp)
    except:
        YAKE_list.append('')


100%|██████████| 216/216 [00:00<00:00, 272.04it/s]


In [ ]:
YAKE_list

[[],
 ['최소한', '미루다'],
 ['아파트'],
 ['핸드폰', '아니다'],
 ['트러블'],
 ['트러블'],
 ['인하다'],
 ['따르다'],
 [],
 ['이해력'],
 [],
 ['상대평가', '전문성', '숙련도'],
 ['상대평가'],
 ['상대평가', '전문성', '숙련도'],
 ['상대평가'],
 ['상대평가', '전문성', '숙련도'],
 ['상대평가', '전문성', '숙련도'],
 ['상대평가', '생크림'],
 ['상대평가'],
 ['상대평가', '전문성', '숙련도'],
 ['상대평가'],
 ['상대평가'],
 ['상대평가', '전문성', '숙련도'],
 ['상대평가'],
 [],
 ['배우다'],
 ['지속적'],
 ['전문성'],
 ['지속적'],
 ['전문성'],
 [],
 [],
 ['지속적'],
 ['더디다'],
 ['자발적'],
 ['초창기', '파트장'],
 ['정확성', '자발적', '문제해결'],
 ['적극성'],
 ['전환배치'],
 ['육아휴직', '인하다'],
 ['인하다', '지원팀'],
 ['인하다', '지원팀'],
 ['인하다', '지원팀'],
 ['운전자', '부정적'],
 ['인하다', '지원팀'],
 ['인하다', '지원팀'],
 ['인하다', '지원팀'],
 ['어리다', '무능력'],
 ['무능력'],
 ['인하다', '지원팀'],
 ['인하다', '지원팀'],
 ['인하다', '육아휴직', '지원팀'],
 ['인하다', '육아휴직', '지원팀'],
 ['인하다', '지원팀'],
 ['인하다', '지원팀'],
 ['인하다', '지원팀'],
 ['무능력'],
 ['초창기', '육아휴직', '전환배치'],
 ['초창기', '파트장'],
 ['초창기', '파트장'],
 ['아니다'],
 ['인하다', '지원팀'],
 ['인하다', '지원팀'],
 ['인하다', '지원팀'],
 ['인하다', '지원팀'],
 ['성희롱'],
 ['인하다', '지원팀'],
 ['솔선수범'],
 ['의하다', '인하다'